In [1]:
import numpy as np
import xarray as xr
import dask 
import os

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial' 
plt.rcParams['font.size'] = 10
plt.rcParams['pdf.fonttype'] = 42

import cartopy.crs as ccrs
import regionmask

### Preliminaries

In [2]:
###################
# Models
###################
from utils import nex_ssp_dict, cil_ssp_dict, isimip_ssp_dict, gardsv_ssp_dict, gardsv_var_dict, deepsdbc_dict

nex_models = list(nex_ssp_dict.keys())
cil_models = list(cil_ssp_dict.keys())
isi_models = list(isimip_ssp_dict.keys())
cbp_gard_models = list(gardsv_ssp_dict.keys())
cbp_gard_precip_models = [model for model in cbp_gard_models if 'pr' in gardsv_var_dict[model]]
cbp_deep_models = list(deepsdbc_dict.keys())

In [13]:
############
# Dask
############
from dask_jobqueue import PBSCluster

cluster = PBSCluster(cores=1, memory='15GB', resource_spec='pmem=15GB',
                     worker_extra_args=['#PBS -l feature=rhel7'], 
                     walltime='00:30:00')

cluster.scale(jobs=20)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.201.237:41323,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Metrics time series plots

In [3]:
###############################
# Set paths
# UPDATE THIS FOR REPRODUCTION
###############################
path_in = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/'
path_out = '/storage/home/dcl5300/work/lafferty-sriver_inprep_tbd/figs/supplementary/'

In [45]:
def make_metric_plot(ensemble, model, ssps, path_in, metric, var_id, path_out):
    out_str = path_out + metric + '/' + var_id + '/' + ensemble + '_' + model + '_' + metric + '_' + var_id + '.png'
    if os.path.isfile(out_str):
        return('Already done')
    else:
        # Figure details
        fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10,6),
                               subplot_kw=dict(projection=ccrs.PlateCarree()))
        axs = axs.flatten()

        fig.subplots_adjust(bottom=0.2, top=0.9, left=0.1, right=0.9,
                    wspace=0.02, hspace=0.01)
        
        fig.suptitle(ensemble + '_' + model + '_' + metric + '_' + var_id, fontsize=12, fontweight='bold', y=0.95)

        # Read netcdf or zarr
        if ensemble in ['cil-gdpcir', 'DeepSD-BC']:
            ds = xr.open_dataset(path_in + ensemble + '/' + metric + '/' + model, engine='zarr')
        elif ensemble == 'isimip3b':
            ds = xr.open_dataset(path_in + metric + '/' + model + '.nc')
        else:
            ds = xr.open_dataset(path_in + ensemble + '/' + metric + '/' + model + '.nc')
        
        # Slice
        ds = ds.sel(lat=slice(-60,90))
        
        # Select variable
        if var_id in list(ds.data_vars):
            ds = ds[var_id]
            
        # Plot by ssp
        for idx, ssp in enumerate(ssps):
            # hard-code the one exception
            if (ensemble == 'DeepSD-BC') and (ssp == 'ssp585') and (var_id == 'pr') and (model == 'MRI-ESM2-0'):
                continue
            else:
                try:
                    p = ds.sel(ssp=ssp, time='2095-12-31').plot(transform=ccrs.PlateCarree(),
                                                                add_colorbar=False,
                                                                ax=axs[idx], levels=11)
                    axs[idx].set_title(ssp)
                    map_exists = True
                except:
                    axs[idx].set_title(ssp + ' ERROR')
                    map_exists = False
        
        # Single colorbar
        if map_exists:
            cbar_ax = fig.add_axes([0.2, 0.16, 0.6, 0.02])

            cbar = fig.colorbar(p, cax=cbar_ax,
                                orientation='horizontal',
                                label=metric + ' ' + var_id)
        
        plt.savefig(out_str, dpi=300)

In [73]:
%%time
################
# avg, max
###############
delayed_res = []
for metric in ['avg', 'max']:
    for var_id in ['tas', 'tasmin', 'tasmax', 'pr']:
        # NEX
        for model in nex_models:
            delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', model, nex_ssp_dict[model], path_in, metric, var_id, path_out))
            
        # CIL
        for model in cil_models:
            delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', model, cil_ssp_dict[model], path_in, metric, var_id, path_out))

        # ISIMIP
        for model in isi_models:
            delayed_res.append(dask.delayed(make_metric_plot)('isimip3b', model, isimip_ssp_dict[model], path_in + 'isimip3b/regridded/conservative/', metric, var_id, path_out))
            
        # GARD-SV
        if var_id == 'pr':
            model_list = cbp_gard_precip_models
        else: 
            model_list = cbp_gard_models
        for model in model_list:
            delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', model, gardsv_ssp_dict[model], path_in + 'carbonplan/regridded/conservative/', metric, var_id, path_out))
        
        # DeepSD-BC
        for model in cbp_deep_models:
            delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', model, deepsdbc_dict[model].keys(), path_in + 'carbonplan/native_grid/', metric, var_id, path_out))
            
################
# max5d
#################
metric = 'max5d'
var_id = 'RX5day'

# NEX
for model in nex_models:
    delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', model, nex_ssp_dict[model], path_in, metric, var_id, path_out))
    
# CIL
for model in cil_models:
    delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', model, cil_ssp_dict[model], path_in, metric, var_id, path_out))

# ISIMIP
for model in isi_models:
    delayed_res.append(dask.delayed(make_metric_plot)('isimip3b', model, isimip_ssp_dict[model], path_in + 'isimip3b/regridded/conservative/', metric, var_id, path_out))
    
# GARD-SV
for model in cbp_gard_precip_models:
    delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', model, gardsv_ssp_dict[model], path_in + 'carbonplan/regridded/conservative/', metric, var_id, path_out))

# DeepSD-BC
for model in cbp_deep_models:
    delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', model, deepsdbc_dict[model].keys(), path_in + 'carbonplan/native_grid/', metric, var_id, path_out))

#############
# dry
#############
metric = 'dry'

for var_id in ['count_eq_0', 'count_lt_1', 'streak_eq_0', 'streak_lt_1']:
    # NEX
    for model in nex_models:
        delayed_res.append(dask.delayed(make_metric_plot)('nex-gddp', model, nex_ssp_dict[model], path_in, metric, var_id, path_out))
        
    # CIL
    for model in cil_models:
        delayed_res.append(dask.delayed(make_metric_plot)('cil-gdpcir', model, cil_ssp_dict[model], path_in, metric, var_id, path_out))

    # ISIMIP
    for model in isi_models:
        delayed_res.append(dask.delayed(make_metric_plot)('isimip3b', model, isimip_ssp_dict[model], path_in + 'isimip3b/regridded/conservative/', metric, var_id, path_out))
        
    # GARD-SV
    for model in cbp_gard_precip_models:
        delayed_res.append(dask.delayed(make_metric_plot)('GARD-SV', model, gardsv_ssp_dict[model], path_in + 'carbonplan/regridded/conservative/', metric, var_id, path_out))
    
    # DeepSD-BC
    for model in cbp_deep_models:
        delayed_res.append(dask.delayed(make_metric_plot)('DeepSD-BC', model, deepsdbc_dict[model].keys(), path_in + 'carbonplan/native_grid/', metric, var_id, path_out))
            
            
# Compute
print(len(delayed_res))
res = dask.compute(*delayed_res)

701
CPU times: user 822 ms, sys: 39.8 ms, total: 862 ms
Wall time: 961 ms
